# Домашнее задание 2 (5 баллов).

*Все задания ниже имеют равный вес (5/10)*

Код для импорта мы написали за вас (не благодарите, нам не трудно). Дальше код будете писать вы. 

[Тут](https://habr.com/ru/companies/ruvds/articles/494720/) шпора по pandas. За основу домашнего задания взят ноутбук [отсюда](https://rutube.ru/video/f884aa6ed5f94120b7304506042fe5bb/) (не подглядывайте!).

In [1]:
import pandas as pd
import numpy as np

#### Описание данных

Автор д/з - плохой человек, который не стал переводить описание с мотивировкой, что весь DS на английском. Так что описание полей будет на английском:

1. Account ID
- Description: A unique identifier for each social media account in the dataset.
- Type: Integer
- Example: 1, 2, 3, …
2. Username
- Description: The username or handle of the social media account.
- Type: String
- Example: john_doe, tech_guru_22, fitness_freak
3. Platform
- Description: The social media platform the account is using (Instagram, Twitter, Facebook, TikTok, LinkedIn).
- Type: Categorical (String)
- Example: Instagram, Twitter, Facebook, TikTok, LinkedIn
4. Follower Count
- Description: The total number of followers the account has.
- Type: Integer
- Example: 1500, 245000, 78000
5. Posts Per Week
- Description: The average number of posts the account creates per week.
- Type: Integer
- Example: 3, 5, 7
6. Engagement Rate
- Description: The percentage of interactions (likes, comments, shares) relative to the follower count. This is a measure of how engaging the content is.
- Type: Float
- Range: 0.01 to 0.15
- Example: 0.045 (4.5% engagement rate)
7. Ad Spend (USD)
- Description: The monthly amount spent on advertising or promoting posts.
- Type: Float
- Example: 150.75, 850.00, 300.50
8. Conversion Rate
- Description: The percentage of users who take a desired action (e.g., clicking a link, signing up, etc.) after interacting with an ad.
- Type: Float
- Range: 0.01 to 0.05 (1% to 5% conversion rate)
- Example: 0.025 (2.5% conversion rate)
9. Campaign Reach
- Description: The total number of unique users reached by the user’s campaigns in a given month.
- Type: Integer
- Example: 5000, 20000, 15000

#### Задание 0

Подгрузите данные. Да-да, за чтение таблицы баллов не будет))

**Hint**: [pd.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)

In [4]:
df = ... # ^⨀ᴥ⨀^

In [2]:
df = pd.read_csv("data.csv")

#### Задание 1

Колонка `Platform` содержит название различных платформ. Давайте представим, что в них есть некоторое отношение порядка. Закодируйте каждую платформу целым числом (от 0 до N) и положите этот "код" в новую колонку `Platform_Code`. Теперь вычислите корреляцию Спирмена между всеми парами колонок в датасете (результатом будет таблица корреляций). В качестве ответа выведите значение корреляции `Platform_Code` с `Engagement Rate`. Можете после вывода числа еще коротко написать, что оно означает (нет, это не оценивается).

**Hint**: [pd.factorize](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.factorize.html), [pd.DataFrame.select_dtypes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.select_dtypes.html), [pd.DataFrame.corr](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html).

In [37]:
# ( ੭ ･ᴗ･ )੭

# Закодируйте каждую платформу целым числом (от 0 до N) и положите этот "код" в новую колонку `Platform_Code`
df['Platform_Code'], unique_platforms = pd.factorize(df['Platform'])

# Теперь вычислите корреляцию Спирмена между всеми парами колонок в датасете (результатом будет таблица корреляций). В качестве ответа выведите значение корреляции `Platform_Code` с `Engagement Rate`
df_corr = df.select_dtypes(include=['number']).corr(method='spearman')
df_corr['Platform_Code']['Engagement Rate'] # близкое к нулю - нет зависимости, больше 0 зависимость прямая, меньше 0 - обратная

0.03138169529349812

#### Задание 2

Теперь посмотрите на столбец `Follower Count`. В нем какие-то числа. Иногда бывает полезно провести дискретизацию такого признака. Разбейте все значения в столбце на 4 группы: "Low", "Medium", "High", "Very High". Каждая группа включает в себя новые 25% данных. То есть, Low включает в себя 25% самых маленьких значений признака и так далее. Положите значения "Low", "Medium", "High" или "Very High" для каждого сэмпла датасета в новую колонку `Follower_Bin`. Теперь посчитайте среднее значение `Engagement Rate` для каждой категории из `Follower_Bin`. В качестве ответа выведите значение для категории "High".

**Hint**: [pd.qcut](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html), [pd.groupby](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html), [pd.DataFrame.mean](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mean.html)

In [38]:
# (◕^^◕)

df['Follower_Bin'] = pd.qcut(df['Follower Count'], 4, labels=['Low', 'Medium', 'High', 'Very High'])
df_group = df.groupby('Follower_Bin', observed=True)['Engagement Rate'].mean()
df_group['High']

0.08655032

#### Задание 3

Иногда бывает полезно превратить широкую таблицу в длинную (например, для визуализаций сразу нескольких признаков на одной картинке). Да, звучит странно, но именно этим вы сейчас и займетесь. Сделайте новый датафрейм `melted_df`, в который вы поместите каждый сэмпл датасета 6 раз: по одному разу на значение из 'Follower Count', 'Posts Per Week', 'Ad Spend (USD)', 'Conversion Rate', 'Engagement Rate' и 'Campaign Reach'. То есть, вы берете сэмпл из датасета (строку) и превращаете ее в 6 отдельных строк. Каждая отдельная строка в столбце `Metric` имеет имя из предложенного списка 5 признаков, а в столбце `Value` - значение данного сэмпла по этому признаку. Значение `Platform` повторяется в этих 6 строках.

Иначе говоря, 

```json
{
    "Account ID": 1,
    "Username": "harrislisa",
    "Platform": "TikTok",
    "Follower Count": 54217,
    "Posts Per Week": 3,
    "Engagement Rate": 0.0986,
    "Ad Spend (USD)": 538.1,
    "Conversion Rate": 0.049,
    "Campaign Reach": 1308,
    "Platform_Code": 0,
    "Follower_Bin": "Low"
}
```

превращается в 

```json
{
    "Platform": "TikTok",
    "Metric": "Follower Count",
    "Value": 54217,
},
{
    "Platform": "TikTok",
    "Metric": "Posts Per Week",
    "Value": 3,
}, ...
```

Для каждого уникальной пары значений (`Platform`, `Metric`) посчитайте моду среди всех значений `Value` для этой пары, результат сделайте списком и оставьте только наибольшее. В качестве ответа выведите сумму полученных мод (сумму всех значений в столбце `Value` уже после вычисления мод). Иначе говоря, выведите сумму всех мод значений для всех уникальных пар (`Platform`, `Metric`).

**Hint**: [pd.melt](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html), [pd.DataFrame.mode](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mode.html), [pd.DataFrameGroupBy.agg](https://pandas.pydata.org/docs/dev/reference/api/pandas.core.groupby.DataFrameGroupBy.agg.html)

In [183]:
# (づ๑•ᴗ•๑)づ♡

df_melt = df.melt(id_vars='Platform', value_vars=['Follower Count', 'Posts Per Week', 'Ad Spend (USD)', 'Conversion Rate', 'Engagement Rate', 'Campaign Reach'], var_name='Metric', value_name='Value')
df_group = df_melt.groupby(['Platform', 'Metric'])['Value']
modes = df_group.apply(lambda x: x.mode())
max_mode = modes.groupby(level=[0,1]).max()
max_mode.sum()

3100285.4716

#### Задание 4

А теперь хочется посмотреть на самые популярные аккаунты на разных платформах. Для каждой платформы отсортируйте датафрейм по убыванию количества подписчиков (`Follower Count`) - да, без циклов, сразу для всех платформ сделать сортировку, а затем оставьте только первые три записи для каждой платформы - это и будут три самых популярных аккаунта для каждой платформы. В качестве ответа выведите саму таблицу и минимальное значение `Follower Count` в ней.

**Hint**: к *groupby* можно применять функции - это эквивалентно применению функции к каждой "группе" внутри groupby-объекта. Читайте [про применение apply к датафрейму после groupby](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#flexible-apply).

In [65]:
# ε(´סּ︵סּ`)з

top_three = df.groupby('Platform')['Follower Count'].apply(lambda x: x.sort_values(ascending=False).head(3))
print(top_three)

top_three.min()


Platform       
Facebook   2403    999982
           7350    997915
           1689    997512
Instagram  8685    999726
           3965    999351
           2189    997844
LinkedIn   3039    999055
           6359    998968
           2159    998925
TikTok     5838    999739
           4234    999234
           2575    998623
Twitter    4920    999919
           9684    999442
           7576    998216
Name: Follower Count, dtype: int64


997512

#### Задание 5

Хочется посчитать какую-то метрику. Мы хотим посмотреть, на отношение разности суммы подписчиков аккаунтов с высокой и низкой конверсией к суммарному охвату рекламы на каждой платформе. То есть, мы делим аккаунты на две группы: высокая и низка конверсия. Затем мы смотрим на то, на сколько сильно влияние аккаунтов с высокой конверсией по сравнению с аккаунтами с низкой конверсией. 

Давайте определим *Conversion Influence* следущим образом:

$$Conversion Influence = \frac{Total Follower\ Count (High) - Total Follower\ Count (Low)}{Total Campaign Reach (High)+Total Campaign Reach (Low)}$$

Считать эту метрику мы будет для каждой `Platform`. В этой формуле High - это значения всех сэмплов датасета, в которых `Conversion Rate` больше медианы, а `Low` - не более медианы. `Total Feature` - это суммарное количество значений `Feature` либо по `High` сэмплам, либо по `Low`.

Чтобы постоянно не пересчитывать, где High. где Low, сделайте новую колонку в датасете `Conversion_Category`. Положите в нее для каждой строки либо High, либо Low.

Выведите платформу с самым большим `Conversion Influence`.

**Hint**: данное задание не про *groupby*, а скорее про [pd.pivot_table](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html). Сделайте сводную таблицу, по которой уже можно посчитать суммы, а затем подставить их в формулы.

In [76]:
 # (︶ω︶)

df['Conversion_Category'] = df['Conversion Rate'].apply(lambda x: 'High' if x > df['Conversion Rate'].median() else 'Low')
pivot = pd.pivot_table(df, index='Platform', columns='Conversion_Category', values=['Follower Count', 'Campaign Reach'], aggfunc='sum')
conversion_influence = (pivot['Follower Count', 'High'] - pivot['Follower Count', 'Low']) / (pivot['Campaign Reach', 'High'] + pivot['Campaign Reach', 'Low'])
conversion_influence.idxmax()

'Twitter'

#### Задание 6

Мы знаем, что вам понравилось считать метрики по формуле. Давайте закрепим этот успех. Теперь для каждой платформы посчитаем, на сколько эффективна реклама в разрезе трех последовательных записей в датасете. 

Для каждой платформы отсортируйте записи в порядке убывания `Posts Per Week`. Будто бы аккаунты, которые постят чаще, используют более "активные" стратегии по рекламе. Теперь посчитайте *скользущие суммы с окном 3* по `Campaign Reach` и `Ad Spend (USD)`. Скользящая сумма с окном N - это вы идете по массиву, берете все последовательные тройки записей и суммируете их. Для первых двух записей троек не найдется. Для них скользящее среднее - NaN, что нам не помешает. 

Теперь для каждого окна посчитайте 

$$Rolling Efficiency Ratio = \frac{Rolling Sum of Campaign Reach}{Rolling Sum of Ad Spend}$$

По сути, для каждого окна вы посчитаете сколько пользователе привлеклось за один доллар, потреченный на рекламу, в данном окне. Понятно, что значений будет столько, сколько окон. Нам интересно максимально значение такой эффективности для каждой платформы.

В качестве ответа выведите название платформы с наибольшей максимальной эффективность и наименьшей (два названия, не одно, не три, ровно два).

**Hint**: окна можно делать через [pd.DataFrame.rolling](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html).

In [102]:
# (◔/‿\◔)

df_sort = df.sort_values(by=['Platform', 'Posts Per Week'], ascending=[True, False])
df_sort['CampaignReachRollingSum'] = df_sort.groupby('Platform')['Campaign Reach'].rolling(window=3).sum().reset_index(level=0, drop=True)
df_sort['AdSpendRollingSum'] = df_sort.groupby('Platform')['Ad Spend (USD)'].rolling(window=3).sum().reset_index(level=0, drop=True)
df_sort['RollingEfficiencyRatio'] = df_sort['CampaignReachRollingSum'] / df_sort['AdSpendRollingSum']
rolling_efficiency_ratio = df_sort.groupby('Platform')['RollingEfficiencyRatio'].max()
print("max: ", rolling_efficiency_ratio.idxmax(), "min: ", rolling_efficiency_ratio.idxmin())

max:  Facebook min:  TikTok


#### Задание 7

Это еще не все прекрасные функции pandas, которые мы хотим вам показать. Теперь вы посчитаете, сколько аккаунтов на каждой платформе одновременно лучшие по `Engagement Rate` и `Conversion Rate`.

Сделайте два отдельных суб-сета. В одном оставьте для каждой платфмормы один топовый аккаунт по `Engagement Rate`, в другом - по `Conversion Rate`. Соедините эти два подмножества по столбцу `Platform` так, что в одно строке есть описание сразу двух аккаунтов-лидеров. Теперь посмотрите равны ли имена аккаунтов в одной строке. Выведите количество строк, в которых названия аккаунтов совпадают.

In [133]:
# ( ͡° ͜ʖ ͡°)

df_engagement = df.loc[df.groupby('Platform')['Engagement Rate'].idxmax()]
df_conversion  = df.loc[df.groupby('Platform')['Conversion Rate'].idxmax()]

# df_conversion = df.groupby('Platform')['Conversion Rate'].nlargest(1)
df_merge = pd.merge(df_engagement[['Platform', 'Username']], df_conversion[['Platform', 'Username']], on='Platform', suffixes=('_engagement', '_conversion'))
count = (df_merge['Username_engagement'] == df_merge['Username_conversion']).sum()
count

0

#### Задание 8

Давайте теперь что-то попроще сделаем. Например, посчитаем отношение суммарного количества подписчиков на аккаунтах с высокой конверсией к такой же сумме в аккаунтах с низкой конверсией (очевидно, для каждой платформы). По сути, мы просто хотим получить число, которое характеризует, на сколько сильно аккаунты с высокой конверсией "доминируют" над аккаунтами с низкой конверсией в плане количества подписчиков.

Высокой конверсией будем считать конверсию больше средней. Остальное - низкая. Посчитайте суммы подписчиков для каждой платформы, поделите одно на другое и выведите разницу между самым большим значением и самым маленьким, а также платформы, которые соотвутствуют этим значениям.

Используйте магическую команду `%%time`, чтобы замерить, сколько времени ушло на исполнение вашего pandas-скрипта.

In [178]:
%%time
# (◡‿◡✿)

df['Conversion_Category'] = df['Conversion Rate'].apply(lambda x: 'High' if x > df['Conversion Rate'].mean() else 'Low')
pivot = pd.pivot_table(df, index='Platform', columns='Conversion_Category', values='Follower Count', aggfunc='sum')
pivot['Ratio'] = pivot['High'] / pivot['Low']
pivot['Ratio'].idxmax()
print(pivot['Ratio'].max() - pivot['Ratio'].min())
print(pivot['Ratio'].idxmax(), pivot['Ratio'].idxmin())

0.17688741338715763
Twitter Instagram
CPU times: total: 406 ms
Wall time: 400 ms


#### Задание 9

А теперь решите задание 8 чисто питоном. Никаких функций и методов pandas. Только питоновские циклы. Замерьте время выполнения кода. Наконец, сравните время в задании 8 и 9. Напишите ниже, кто же победил: чистый python и pandas?

**Hint**: Чтобы итерироваться по датафрейму, можно из него сделать генератор через [pd.DataFrame.iterrows](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iterrows.html) или [pd.DataFrame.itertuples](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.itertuples.html#pandas.DataFrame.itertuples). К слову, это не все способы итерироваться по датафрейму.

In [179]:
%%time
# (✿◠‿◠)

mean = df['Conversion Rate'].mean()
dict = {}
for row in df.itertuples():
    if row._8 > mean:
        category = 'High'
    else:
        category = 'Low'
    dict[(row.Platform, category)] = dict.get((row.Platform, category), 0) + row._4

ratios = {}
platforms = set()
for (platform, category), sum_followers in dict.items():
    platforms.add(platform)

for platform in platforms:
    high = dict.get((platform, 'High'), 0)
    low  = dict.get((platform, 'Low'), 0)
    ratio = high / low
    ratios[platform] = ratio

max_key = None
min_key = None
max_val = None
min_val = None
for k, v in ratios.items():
    if max_val is None or v > max_val:
        max_val = v
        max_key = k
    if min_val is None or v < min_val:
        min_val = v
        min_key = k
print(max_val - min_val)
print(max_key, min_key)


0.17688741338715763
Twitter Instagram
CPU times: total: 15.6 ms
Wall time: 22.8 ms


**А победителем является**: python

#### Задание 10

Крайне серьезное задание. Отнеситесь к нему соответствующе. В ячейке ниже напишите ваш любимый анекдот или мем (только без баянов, окей?). Можно плохие. Помните, это задание на полный балл. Проверяющий работу ассистент должен улыбнуться.

Если вставляете картинку, то убедитесь, что вы ее не подгружаете локально. А то будет неудобно - потерять балл на этом задании, когда надо было выложить картинку на облако и прокинуть ссылку. И нет, нельзя сюда просто ссылку вставить. Либо ищите, как вставить картинку, либо смешной анекдот. Есть всего два стула - выбирайте...

# ‿( ́ ̵ _-`)‿
![Моя картинка](https://cdn.discordapp.com/attachments/679990387544293438/1340788458519531550/IMG_20210826_005212.png?ex=67b3a20f&is=67b2508f&hm=6a72ad025630c1378ae18ab2241a03de733ca71a779b810069eaef5391bdd5a0&)
